# 2.0.x: Add AOA fractional cover to multires stats

Another useful metric is each model's fractional AOA, or 1 minus the fraction of pixels that are outside of the AOA. We can add this to the multires stats dataframe for each model as well.

## Imports and config

In [1]:
from pathlib import Path

import pandas as pd
from dask import compute, delayed

from src.conf.conf import get_config
from src.conf.environment import log
from src.utils.dask_utils import close_dask, init_dask
from src.utils.dataset_utils import get_aoa_dir
from src.utils.raster_utils import open_raster

cfg = get_config()

Gather filenames

In [2]:
splot_fns = [
    list(Path(d, "splot").glob("*.tif"))[0]
    for d in sorted(list(get_aoa_dir(cfg).glob("*")))
    if d.is_dir()
]

comb_fns = [
    list(Path(d, "splot_gbif").glob("*.tif"))[0]
    for d in sorted(list(get_aoa_dir(cfg).glob("*")))
    if d.is_dir()
]

Compute the AOA fractions

In [3]:
@delayed
def aoa_frac(fn: Path) -> tuple[str, float]:
    ds = open_raster(fn).sel(band=2)
    frac = 1 - (ds == 1).sum().values / (ds == 0).sum().values
    ds.close()
    del ds
    return fn.parents[1].stem, frac

# Initalize dask
client, cluster = init_dask(dashboard_address=cfg.dask_dashboard)

splot_aoa_fracs = compute(*[aoa_frac(fn) for fn in splot_fns])
comb_aoa_fracs = compute(*[aoa_frac(fn) for fn in comb_fns])

# Close dask
close_dask(client, cluster)

Append them to the multires stats dataframe

In [7]:
all_results = pd.read_parquet("results/all_results.parquet")

# splot_aoa = pd.DataFrame(splot_aoa_fracs, columns=["trait_id", "aoa"])
# comb_aoa = pd.DataFrame(comb_aoa_fracs, columns=["trait_id", "aoa"])

for trait_id, aoa in splot_aoa_fracs:
    rows = all_results.query(
        f"trait_id == '{trait_id}' and "
        "trait_set == 'splot' and "
        f"resolution == '{cfg.model_res}'"
    )
    all_results.loc[rows.index, "aoa"] = aoa

for trait_id, aoa in comb_aoa_fracs:
    rows = all_results.query(
        f"trait_id == '{trait_id}' and "
        "trait_set == 'splot_gbif' and "
        f"resolution == '{cfg.model_res}'"
    )
    all_results.loc[rows.index, "aoa"] = aoa

In [12]:
all_results.query(f"resolution == '{cfg.model_res}'")[["trait_id", "trait_set", "aoa"]]

,trait_id,trait_set,aoa
0,X163_mean,splot,0.905965
1,X163_mean,splot,0.905965
2,X163_mean,gbif,NaN
3,X163_mean,gbif,NaN
4,X163_mean,splot_gbif,0.993581
...,...,...,...
193,X144_mean,splot,0.977485
194,X144_mean,gbif,NaN
195,X144_mean,gbif,NaN
196,X144_mean,splot_gbif,0.995805
